[🥭 Entropy Markets](https://entropy.markets/) support is available at: [Docs](https://docs.entropy.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/entropymarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=Withdraw.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/entropy-explorer-examples/HEAD?labpath=Withdraw.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Withdraw

This code will connect to the _mainnet_ cluster and withdraw 0.1 SOL from a Entropy `Account`.

Unlike `deposit()`, you specify an address as the destination. You can withdraw from your Entropy `Account` to anyone's wallet (be careful!). Also, you can withdraw tokens even if you don't own them (as long as you have sufficient collateral) by borrowing them - `withdraw()`'s `allow_borrow` boolean allows or prohibits borrowing tokens the `Account` doesn't own.

In [ ]:
import decimal
import entropy

from solana.publickey import PublicKey


# Use our hard-coded mainnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file. This wallet
# has a Entropy Account at AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB.
wallet = entropy.Wallet(bytes([181,213,227,47,41,229,109,138,15,82,26,7,230,184,88,102,197,215,238,155,136,196,138,92,98,154,67,68,47,140,90,40,248,149,223,193,241,51,4,196,126,32,211,66,90,137,249,160,132,246,38,29,88,16,252,116,12,83,117,158,40,98,178,54]))

with entropy.ContextBuilder.build(cluster_name="mainnet") as context:
    group = entropy.Group.load(context)
    account = entropy.Account.load(context, PublicKey("AX7P7YXUNj7Ycp5VG5tweZqYngTk4kNFNZ7Zd2XHDrxB"), group)
    sol_token = entropy.token(context, "SOL")

    print("Wrapped SOL in account", account.slot_by_instrument(sol_token).net_value)

    withdraw_value = entropy.InstrumentValue(sol_token, decimal.Decimal("0.1"))
    withdrawal_signatures = account.withdraw(context, wallet, wallet.address, withdraw_value, False)
    print("Waiting for withdraw transaction to confirm...", withdrawal_signatures)
    entropy.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, withdrawal_signatures, commitment="processed"
        )

    account = entropy.Account.load(context, account.address, group)
    print("Wrapped SOL after withdrawal", account.slot_by_instrument(sol_token).net_value)

print("Example complete.")